In [ ]:
# packages and dependencies import
import os
import sys
import json
import pandas as pd # used for data manipulation
import numpy as np # used for mathematical operations

# Used for Exploratory Data Analysis
import seaborn as sns 

# used for data visualization
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# Used for data transfer and visualization 
import plotly.graph_objects as go
import plotly.io as pio

In [ ]:
# Data Import
data = pd.read_csv("/home/tjselevani/Desktop/Apps/vscode/python/python analysis/data/last-3-months-transactions.csv")

# Convert 'created_at' column to datetime if not already
data['created_at'] = pd.to_datetime(data['created_at'])

# Extract the week from 'created_at'
data.loc[:, 'week'] = data['created_at'].dt.to_period('W').astype(str)

In [ ]:

def format_weeks_xticks(week_labels):
    """Returns tick positions and labels for weeks."""
    return range(len(week_labels)), week_labels

In [ ]:
data = data[data['transaction_type'] == 'CREDIT'].copy()

In [ ]:
# Define unique colors for each vehicle, plus "Unknown" and "Failed Transactions"
vehicle_colors = {
    'SM191': 'blue',
    'SM192': 'green',
    'SM944': 'yellow',
    'SM055': 'purple',
    'SM024': 'orange',
    'Unknown': 'gray',
    'Failed': 'red'
}

# Map colors to vehicle_booked, filling NaN values with a default color (e.g., gray)
data.loc[:,'color'] = data['vehicle_booked'].map(vehicle_colors).fillna('gray')

# Define vehicles list (unique vehicles from the data)
vehicles = data['vehicle_booked'].dropna().unique()

In [ ]:
# Get all unique weeks in chronological order
all_weeks = sorted(data['week'].unique())

In [ ]:
# Aggregate data by week for each vehicle
weekly_by_vehicle = data.groupby(['week', 'vehicle_booked'])['amount'].sum().reset_index()

# Pivot the data to have vehicles as columns
weekly_pivot = weekly_by_vehicle.pivot(index='week', columns='vehicle_booked', values='amount').reset_index()

# Sort by week (assuming format is YYYY-MM)
weekly_pivot = weekly_pivot.sort_values('week')

In [ ]:
# plot_weekly_fares_line(data, "weekly_total_earnings_trend_line", vehicles)
# plot_weekly_total_revenue(data, "weekly_total_earnings_bar")
# plot_weekly_breakdown_by_vehicle("weekly_stacked_earnings")

In [ ]:
def plot_weekly_fares_line(output_file, vehicles=None):

    # If no vehicles are provided, use all unique vehicles from the data
    if vehicles is None:
        vehicles = data['vehicle_booked'].dropna().unique()

    # Aggregate total fare for each vehicle by week
    weekly_vehicle_fares = data.groupby(['week', 'vehicle_booked'])['amount'].sum().reset_index()
    
    # Create a pivot table for better visualization
    pivot_data = weekly_vehicle_fares.pivot(index='week', columns='vehicle_booked', values='amount').fillna(0)
    
    # Ensure all vehicles are present
    for vehicle in vehicles:
        if vehicle not in pivot_data.columns:
            pivot_data[vehicle] = 0
    
    # Reindex with all weeks to ensure chronological order and fill missing weeks
    all_weeks = sorted(data['week'].unique())
    pivot_data = pivot_data.reindex(all_weeks).fillna(0)
    
    plot_data = go.Figure()

    for vehicle in vehicles:
        if vehicle in pivot_data.columns:
            plot_data.add_trace(go.Scatter(
                x=all_weeks,
                y=pivot_data[vehicle],
                mode='lines+markers',
                name=vehicle,
                line=dict(width=2),
                marker=dict(symbol='circle', size=6),
                text=pivot_data[vehicle].apply(lambda x: f"{int(x):,}"),
                textposition='top center'
            ))

    plot_data.update_layout(
        title="Weekly Fare Trends by Vehicle",
        xaxis_title="Week",
        yaxis_title="Total Fare Amount (KSH)",
        xaxis_tickangle=-45,
        template='plotly_white'
    )

    # Save Plotly JSON
    plot_data_json = plot_data.to_json()

    # Create output directories
    json_dir = f"../json/all/week"
    os.makedirs(json_dir, exist_ok=True)

    json_path = f"{json_dir}/{output_file}.json"
    with open(json_path, 'w') as json_file:
        json.dump(plot_data_json, json_file)

    print(f"Saved JSON Plotly data: {json_path}")

# plot_weekly_fares_line("weekly_total_earnings_trend_line", vehicles)


In [ ]:
def plot_weekly_total_revenue(output_file):
    # Calculate total revenue per week
    weekly_total = data.groupby('week')['amount'].sum().reset_index()
    weekly_total = weekly_total.sort_values('week')

    # Plot using Plotly for JSON output
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=weekly_total['week'],
        y=weekly_total['amount'],
        marker=dict(color='skyblue'),
        text=weekly_total['amount'].apply(lambda x: f"{int(x):,}"),
        textposition='auto',
        name="Total Earnings"
    ))

    # Update layout for the Plotly plot
    fig.update_layout(
        title="Weekly Total Earnings - All Vehicles",
        xaxis_title="Week",
        yaxis_title="Total Earnings (KSH)",
        xaxis_tickangle=-45,
        template='plotly_white'
    )

    # Save the Plotly JSON
    fig_json = fig.to_json()

    # Create output directories
    json_dir = f"../json/all/week"
    os.makedirs(json_dir, exist_ok=True)

    json_path = f"{json_dir}/{output_file}.json"
    with open(json_path, 'w') as json_file:
        json.dump(fig_json, json_file)

    print(f"Saved JSON Plotly data: {json_path}")

    # Show the plot using Plotly
    fig.show()

# Example usage
# plot_weekly_total_revenue("weekly_total_earnings_bar")


In [ ]:
def plot_weekly_breakdown_by_vehicle(output_file):
    
    # Aggregate total earnings by week and vehicle
    weekly_by_vehicle = data.groupby(['week', 'vehicle_booked'])['amount'].sum().reset_index()

    # Create a pivot table for better visualization
    weekly_by_vehicle_pivot = weekly_by_vehicle.pivot_table(index='week', columns='vehicle_booked', values='amount')
    weekly_by_vehicle_pivot = weekly_by_vehicle_pivot.fillna(0)  # Replace NaN with 0
    weekly_by_vehicle_pivot = weekly_by_vehicle_pivot.sort_index()  # Sort by week

    # Create the stacked bar chart using Matplotlib
    plt.figure(figsize=(16, 8))
    
    weekly_by_vehicle_pivot.plot(kind='bar', stacked=True, figsize=(16, 8), colormap='viridis')

    plt.title("Weekly Earnings Breakdown by Vehicle", fontsize=16)
    plt.xlabel("Week", fontsize=12)
    plt.ylabel("Total Earnings (KSH)", fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.4)
    plt.legend(title="Vehicles", bbox_to_anchor=(1.05, 1), loc='upper left')

    # Add total value annotations on top of the bars
    weekly_totals = weekly_by_vehicle_pivot.sum(axis=1)
    for i, total in enumerate(weekly_totals):
        plt.text(i, total + (total*0.01), f"Total: {int(total):,}", ha='center', va='bottom', fontsize=9, fontweight='bold')

    plt.tight_layout()

    # Save the plot as an image using Matplotlib
    plt.savefig(output_file, dpi=300)

    # Prepare data for Plotly-compatible JSON format
    plot_data = {
        'data': [],
        'layout': {
            'title': 'Weekly Earnings Breakdown by Vehicle',
            'xaxis': {'title': 'Week'},
            'yaxis': {'title': 'Total Earnings (KSH)'},
            'barmode': 'stack',
            'legend': {'title': 'Vehicles'}
        }
    }

    # Convert weekly_by_vehicle_pivot to Plotly-compatible format
    for vehicle in weekly_by_vehicle_pivot.columns:
        plot_data['data'].append({
            'x': weekly_by_vehicle_pivot.index.tolist(),
            'y': weekly_by_vehicle_pivot[vehicle].tolist(),
            'type': 'bar',
            'name': vehicle
        })

    # Create output directories
    json_dir = f"../json/all/week"
    os.makedirs(json_dir, exist_ok=True)

    json_path = f"{json_dir}/{output_file}.json"
    with open(json_path, 'w') as json_file:
        json.dump(plot_data, json_file)

    print(f"Saved JSON Plotly data: {json_path}")


# Example usage
# plot_weekly_breakdown_by_vehicle("weekly_stacked_earnings")
